In [15]:
import mne
import math
import numpy as np
import neurokit2 as nk
import pandas as pd
import pickle
import random

from scipy.io import loadmat
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from os.path import exists


In [16]:
participants = 89
excluded_numbers = [8, 35, 39, 81]
label_file_path='./details/labels.csv'
subjects = [idx for idx in range(1, participants+1) if idx not in excluded_numbers]
random.shuffle(subjects)
subjects

[80,
 70,
 52,
 72,
 7,
 55,
 2,
 64,
 17,
 67,
 44,
 59,
 89,
 32,
 73,
 79,
 43,
 21,
 9,
 56,
 6,
 12,
 76,
 26,
 78,
 53,
 30,
 82,
 10,
 71,
 77,
 14,
 58,
 50,
 68,
 18,
 16,
 5,
 47,
 4,
 40,
 33,
 37,
 24,
 23,
 25,
 42,
 34,
 41,
 63,
 60,
 46,
 45,
 61,
 49,
 54,
 85,
 19,
 57,
 3,
 84,
 66,
 20,
 29,
 38,
 11,
 15,
 62,
 28,
 51,
 87,
 75,
 83,
 22,
 69,
 86,
 13,
 65,
 88,
 74,
 48,
 31,
 27,
 36,
 1]

In [17]:
def load_data(subject):
    if exists('./eeg_data/{}.mat'.format(subject)):
        file='./eeg_data/{}.mat'.format(subject)
        mat_data = loadmat(file)
        eeg_data = (mat_data['EEG']['data'][0, 0])*1e-6
        channel_names = [ch[0] for ch in mat_data['EEG']['chanlocs'][0, 0]['labels'][0]]
        sampling_freq = mat_data['EEG']['srate'][0, 0][0, 0]
        return eeg_data,channel_names,sampling_freq

In [18]:
def creating_mne_and_filtering(eeg_data,channel_names,sampling_freq,high_pass=0,low_pass=45):
    fp1_index = channel_names.index('FP1')
    fp2_index = channel_names.index('FP2')
    selected_channels_data = eeg_data[[fp1_index, fp2_index], :]
    info = mne.create_info(['FP1', 'FP2'], sampling_freq, ch_types='eeg')
    raw = mne.io.RawArray(selected_channels_data, info)
    raw.filter(high_pass,low_pass)
    df=raw.get_data()
    return df

In [19]:
def initializing_labels(eeg_data,subject_number, label_file_path=label_file_path,healthy_range=15):
    df = pd.read_csv(label_file_path)
    df.dropna(subset=['Tension (1, 8, 15, 21, 28, and 35)'], inplace=True)
    if subject_number in df['ID'].values:
        tension_value = df.loc[df['ID'] == subject_number, 'Tension (1, 8, 15, 21, 28, and 35)'].values[0]
        if tension_value<=healthy_range:
            data=(eeg_data,0)
            return data
        else:
            data=(eeg_data,1)
            return data
    else:
        return None

In [20]:
def sliding_window_augmentation(data, window_size, stride):
    data=np.array(data)
    _,n_timepoints,  = data.shape
    augmented_data = []
    for start in range(0, n_timepoints - window_size + 1, stride):
        end = start + window_size
        segment = data[:,start:end]
        augmented_data.append(segment)
    return np.array(augmented_data)

In [21]:
def creating_normalized_chunks(eeg_tuple,sampling_freq=250,time=2,healthy_augment_ratio=1,anxiety_augment_ratio=1):
    scaler=MinMaxScaler()
    normalized_data=[]
    data,label=eeg_tuple
    chunk_size=sampling_freq*time
    healhty_stride = chunk_size // healthy_augment_ratio
    anxiety_stride = chunk_size // anxiety_augment_ratio
    n_chunks=data.shape[1]//chunk_size
    data=data[:,:n_chunks*chunk_size]
    if label==0:
        augmented_eeg_data = sliding_window_augmentation(data, chunk_size, healhty_stride)
    else:
        augmented_eeg_data = sliding_window_augmentation(data, chunk_size, anxiety_stride)
    #data_new=np.split(data,n_chunks,axis=1)
    for i in range(len(augmented_eeg_data)):
        x=augmented_eeg_data[i]
        x=x.T
        x=scaler.fit_transform(x)
        x=x.T
        normalized_data.append((x,label))
    return normalized_data
    #normalized_data=np.array(normalized_data)

In [22]:
def baseline_correction(eeg_data,order=6,fp1_index=0,fp2_index=1):
    baseline_corrected_eeg_data=[]
    for eeg_chunk in eeg_data:
        eeg_data,labels=eeg_chunk
        baseline_poly_fp1 = np.polyfit(np.arange(len(eeg_data[fp1_index])), eeg_data[fp1_index], order)  # Fit a 6th order polynomial to the baseline drift for each channel
        baseline_poly_fp2 = np.polyfit(np.arange(len(eeg_data[fp2_index])), eeg_data[fp2_index], order)
        baseline_drift_fp1 = np.polyval(baseline_poly_fp1, np.arange(len(eeg_data[fp1_index])))
        baseline_drift_fp2 = np.polyval(baseline_poly_fp2, np.arange(len(eeg_data[fp2_index])))   # Evaluate the polynomials at all time points to get the baseline drifts
        eeg_data[fp1_index] -= baseline_drift_fp1   # Subtract the baseline drifts from the original signals
        eeg_data[fp2_index] -= baseline_drift_fp2
        baseline_corrected_eeg_data.append((eeg_data,labels))
    return baseline_corrected_eeg_data

In [23]:
def fft(eeg_chunks):
    fft_features=[]
    for eeg_sample in eeg_chunks:
        data,label=eeg_sample
        segg=[]
        for i in range(len(data)):
            fourier = np.abs(np.fft.fft(data[i, :]))
            segg.append(fourier)
        final_concate = np.concatenate(segg, axis=0)
        fft_features.append((final_concate,label))
    return fft_features

In [24]:
def manual_features_extraction(data):
    return np.concatenate((
        np.mean(data, axis=-1),np.std(data, axis=-1),
        np.ptp(data, axis=-1),
        np.var(data, axis=-1),
        np.min(data, axis=-1),
        np.max(data, axis=-1),
        np.argmin(data, axis=-1),
        np.argmax(data, axis=-1),
        np.mean(data**2, axis=-1),
        np.sqrt(np.mean(data**2, axis=-1)),
        np.sum(np.abs(np.diff(data, axis=-1)), axis=-1),
        stats.skew(data, axis=-1),
        stats.kurtosis(data, axis=-1)
    ), axis=-1)


In [25]:
def manual_features(eeg_chunks):
    manual_feature=[]
    for eeg_sample in eeg_chunks:
        data,labels=eeg_sample
        manual_feature.append((manual_features_extraction(data),labels))
    return manual_feature

In [26]:
def power_range_features(eeg_chunks,sampling_freq):
    power_range_data=[]
    for eeg_sample in eeg_chunks:
        data,labels=eeg_sample
        eeg_power=nk.eeg_power(data,sampling_rate=sampling_freq)
        eeg_power_features = eeg_power.set_index('Channel').stack().reset_index()
        eeg_power_features.columns = ['Channel', 'FrequencyBand', 'Power']
        power_values=np.array(eeg_power_features['Power'].values.tolist())
        power_range_data.append((power_values,labels))
    return power_range_data

In [27]:
def preprocessing(subjects,train_percentage=0.7,high_pass=0,low_pass=45,healthy_label_range=15,time=15,healthy_augment_ratio=1,anxiety_augment_ratio=3,order=6,fp1_index=0,fp2_index=1):
    train_limit = math.ceil(train_percentage * participants)     
    fft_class_train = []
    fft_class_test = []
    manual_class_train = []
    manual_class_test = []
    power_class_train = []
    power_class_test = []
    class_train = []
    class_test = []
    limit=1
    for i in subjects:
        eeg_data,channel_names,sampling_freq=load_data(i)
        df=creating_mne_and_filtering(eeg_data,channel_names,sampling_freq,high_pass=high_pass,low_pass=low_pass)
        eeg_tuple=initializing_labels(df,i,healthy_range=healthy_label_range)
        eeg_chunks=creating_normalized_chunks(eeg_tuple,sampling_freq=sampling_freq,time=time,healthy_augment_ratio=healthy_augment_ratio,anxiety_augment_ratio=anxiety_augment_ratio)
        eeg_chunks=baseline_correction(eeg_chunks,order=order,fp1_index=fp1_index,fp2_index=fp2_index)
        fft_chunk=fft(eeg_chunks)
        manual_chunks=manual_features(eeg_chunks)
        power_range_chunks=power_range_features(eeg_chunks,sampling_freq)
        if limit<train_limit:
            fft_class_train+=fft_chunk
            manual_class_train+=manual_chunks
            power_class_train+=power_range_chunks
            class_train+=eeg_chunks
        else:
            fft_class_test+=fft_chunk
            manual_class_test+=manual_chunks
            power_class_test+=power_range_chunks
            class_test+=eeg_chunks
        limit=limit+1
    return fft_class_train,fft_class_test,manual_class_train,manual_class_test,power_class_train,power_class_test,class_train,class_test
        

In [28]:
fft_class_train,fft_class_test,manual_class_train,manual_class_test,power_class_train,power_class_test,class_train,class_test=preprocessing(subjects,train_percentage=0.7,high_pass=0,low_pass=45,healthy_label_range=15,time=2,healthy_augment_ratio=1,anxiety_augment_ratio=3,order=6,fp1_index=0,fp2_index=1)

Creating RawArray with float64 data, n_channels=2, n_times=1171459
    Range : 0 ... 1171458 =      0.000 ...  4685.832 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 75 samples (0.300 s)

Creating RawArray with float64 data, n_channels=2, n_times=1170119
    Range : 0 ... 1170118 =      0.000 ...  4680.472 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 d

In [29]:
pd.DataFrame(class_train[0][0])

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,-0.015062,-0.074966,-0.026167,0.065336,0.050232,-0.075197,-0.144014,-0.050299,0.089051,0.085837,...,0.053471,-0.056873,0.026827,0.215407,0.269849,0.133248,0.012506,0.095172,0.291233,0.357123
1,0.056291,-0.063272,-0.046341,0.043118,0.035482,-0.089573,-0.162090,-0.064421,0.087518,0.093965,...,0.059297,-0.054889,0.030356,0.224607,0.280401,0.136762,0.006590,0.085897,0.285269,0.354383


In [30]:
pd.DataFrame(fft_class_train[0][0])

,0
0,2.499334e-12
1,6.729928e-02
2,3.470808e+00
3,1.053287e+01
4,1.344638e+01
...,...
995,1.230858e+01
996,1.288582e+01
997,1.055673e+01
998,3.444439e+00


In [31]:
pd.DataFrame(fft_class_train)[1].value_counts()

1
0    106536
1    100496
Name: count, dtype: int64

In [32]:
a,b=pd.DataFrame(fft_class_train)[1].value_counts().tolist()
print(int((b/a)*100),'%')

94 %


In [33]:
pd.DataFrame(fft_class_train[0][0])

,0
0,2.499334e-12
1,6.729928e-02
2,3.470808e+00
3,1.053287e+01
4,1.344638e+01
...,...
995,1.230858e+01
996,1.288582e+01
997,1.055673e+01
998,3.444439e+00


In [35]:
import os

# 检查samples文件夹是否存在
samples_folder = './samples'
if not os.path.exists(samples_folder):
    # 如果samples文件夹不存在，则创建它
    os.makedirs(samples_folder)

with open('./samples/fft_class_train.pkl', 'wb') as f:
    pickle.dump(fft_class_train, f)

with open('./samples/fft_class_test.pkl', 'wb') as f:
    pickle.dump(fft_class_test, f)


with open('./samples/manual_class_train.pkl', 'wb') as f:
    pickle.dump(manual_class_train, f)

with open('./samples/manual_class_test.pkl', 'wb') as f:
    pickle.dump(manual_class_test, f)


with open('./samples/power_class_train.pkl', 'wb') as f:
    pickle.dump(power_class_train, f)

with open('./samples/power_class_test.pkl', 'wb') as f:
    pickle.dump(power_class_test, f)   


with open('./samples/class_train.pkl', 'wb') as f:
    pickle.dump(class_train, f)

with open('./samples/class_test.pkl', 'wb') as f:
    pickle.dump(class_test, f)